In [1]:
!git clone https://github.com/masa-ita/keras-composer.git

Cloning into 'keras-composer'...
remote: Enumerating objects: 177, done.
remote: Counting objects: 100% (177/177), done.
remote: Compressing objects: 100% (166/166), done.
remote: Total 177 (delta 16), reused 171 (delta 10), pack-reused 0
Receiving objects: 100% (177/177), 1.00 GiB | 16.01 MiB/s, done.
Resolving deltas: 100% (16/16), done.


In [0]:
!mkdir keras-composer/weights

In [0]:
from keras import objectives, backend as K
from keras.layers import Bidirectional, Dense, Embedding, Input, Lambda, CuDNNLSTM, RepeatVector, TimeDistributed, Dropout
from keras.models import Model
import keras
from keras.regularizers import l2

In [0]:
class VAE(object):
    def __init__(self, vocab_size=500, max_length=300, latent_rep_size=200):
        self.encoder = None
        self.decoder = None
        self.autoencoder = None

        x = Input(shape=(max_length, vocab_size))

        vae_loss, encoded = self._build_encoder(x, latent_rep_size=latent_rep_size, max_length=max_length)
        self.encoder = Model(inputs=x, outputs=encoded)
        encoder_out = self.encoder(x)

        encoded_input = Input(shape=(latent_rep_size,))

        decoded = self._build_decoder(encoded_input, vocab_size, max_length)
        self.decoder = Model(encoded_input, decoded)
        
        decoder_out = self.decoder(encoder_out)

        self.autoencoder = Model(inputs=x, outputs=decoder_out)
        self.autoencoder.compile(optimizer='Adam',
                                 loss=vae_loss,
                                 metrics=['accuracy'])
        
    def _build_encoder(self, x, latent_rep_size=200, max_length=300, epsilon_std=0.01):
        h = CuDNNLSTM(500, return_sequences=False, kernel_regularizer=l2(0.0001), recurrent_regularizer=l2(0.0001), name='lstm_1')(x)
        h = Dropout(0.2)(h)
        h = Dense(256, activation='relu', name='dense_1')(h)

        def sampling(args):
            z_mean_, z_log_var_ = args
            batch_size = K.shape(z_mean_)[0]
            epsilon = K.random_normal(shape=(batch_size, latent_rep_size), mean=0., stddev=epsilon_std)
            return z_mean_ + K.exp(z_log_var_ / 2) * epsilon

        z_mean = Dense(latent_rep_size, name='z_mean', activation='linear')(h)
        z_log_var = Dense(latent_rep_size, name='z_log_var', activation='linear')(h)
    
        def vae_loss(x, x_decoded_mean):
            x = K.flatten(x)
            x_decoded_mean = K.flatten(x_decoded_mean)
            xent_loss = max_length * objectives.binary_crossentropy(x, x_decoded_mean)
            kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
            return xent_loss + kl_loss

        return (vae_loss, Lambda(sampling, output_shape=(latent_rep_size,), name='lambda')([z_mean, z_log_var]))

    def _build_decoder(self, encoded, vocab_size, max_length):
        repeated_context = RepeatVector(max_length)(encoded)
    
        h = CuDNNLSTM(500, return_sequences=True, name='dec_lstm_1')(repeated_context)
    
        decoded = TimeDistributed(Dense(vocab_size, activation='softmax'), name='decoded_mean')(h)
    
        return decoded


In [0]:
from keras.callbacks import ModelCheckpoint
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
# from model import VAE
import numpy as np
import os

In [0]:
MAX_LENGTH = 300
NUM_WORDS = 124

In [7]:
!pip install music21

    100% |████████████████████████████████| 18.0MB 709kB/s 
  Running setup.py bdist_wheel for music21 ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done
  Stored in directory: /root/.cache/pip/wheels/53/8b/a6/be1921c60a68f0bea31c6b6a0a7b125badd61294d6a694407f
Successfully built music21


In [0]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord

In [0]:
def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []
    songs = []

    for file in glob.glob("keras-composer/midi_songs/*.mid"):
        song = []
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                song.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                song.append('.'.join(str(n) for n in element.normalOrder))
        songs.append(song)
        notes += song

    with open('keras-composer/data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes, songs


In [10]:
notes, songs = get_notes()

Parsing keras-composer/midi_songs/bwv781.mid
Parsing keras-composer/midi_songs/bwv777.mid
Parsing keras-composer/midi_songs/bwv776.mid
Parsing keras-composer/midi_songs/bwv779.mid
Parsing keras-composer/midi_songs/bwv778.mid
Parsing keras-composer/midi_songs/bwv774.mid
Parsing keras-composer/midi_songs/bwv785.mid
Parsing keras-composer/midi_songs/bwv780.mid
Parsing keras-composer/midi_songs/bwv784.mid
Parsing keras-composer/midi_songs/bwv773.mid
Parsing keras-composer/midi_songs/bwv775.mid
Parsing keras-composer/midi_songs/bwv783.mid
Parsing keras-composer/midi_songs/bwv786.mid
Parsing keras-composer/midi_songs/bwv772.mid
Parsing keras-composer/midi_songs/bwv782.mid


In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

songs_text = [' '.join(song) for song in songs]

tokenizer = Tokenizer(num_words=NUM_WORDS, filters='', lower=False)
tokenizer.fit_on_texts(songs_text)
note2code = tokenizer.word_index

songs_codes = tokenizer.texts_to_sequences(songs_text)
padded_songs = pad_sequences(songs_codes, maxlen=MAX_LENGTH)

In [0]:
temp = np.zeros((padded_songs.shape[0], MAX_LENGTH, NUM_WORDS))
temp[np.expand_dims(np.arange(padded_songs.shape[0]), axis=0).reshape(padded_songs.shape[0], 1), np.repeat(np.array([np.arange(MAX_LENGTH)]), padded_songs.shape[0], axis=0), padded_songs] = 1

songs_one_hot = temp

In [0]:
def create_model_checkpoint(dir, model_name):
    filepath = dir + '/' + \
               model_name + "-{epoch:02d}-{acc:.2f}-{loss:.2f}.h5"
    directory = os.path.dirname(filepath)

    try:
        os.stat(directory)
    except:
        os.mkdir(directory)

    checkpointer = ModelCheckpoint(filepath=filepath,
                                   verbose=1,
                                   save_best_only=False)

    return checkpointer

In [23]:
model = VAE(vocab_size=NUM_WORDS, latent_rep_size=2, max_length=MAX_LENGTH)

checkpointer = create_model_checkpoint('weights', 'music_vae')

history = model.autoencoder.fit(x=songs_one_hot, y=songs_one_hot,
                      batch_size=1, epochs=8000, callbacks=[checkpointer])

Epoch 4367/8000
15/15 [==============================] - 2s 153ms/step - loss: 6.0539 - acc: 0.4842

Epoch 04367: saving model to weights/music_vae-4367-0.48-6.05.h5
Epoch 4368/8000
15/15 [==============================] - 2s 150ms/step - loss: 6.3424 - acc: 0.4393

Epoch 04368: saving model to weights/music_vae-4368-0.44-6.34.h5
Epoch 4369/8000
15/15 [==============================] - 2s 154ms/step - loss: 7.1215 - acc: 0.4109

Epoch 04369: saving model to weights/music_vae-4369-0.41-7.12.h5
Epoch 4370/8000
15/15 [==============================] - 2s 154ms/step - loss: 8.0540 - acc: 0.3200

Epoch 04370: saving model to weights/music_vae-4370-0.32-8.05.h5
Epoch 4371/8000
15/15 [==============================] - 2s 154ms/step - loss: 18.6727 - acc: 0.0411

Epoch 04371: saving model to weights/music_vae-4371-0.04-18.67.h5
Epoch 4372/8000
15/15 [==============================] - 2s 153ms/step - loss: 17.7721 - acc: 0.0416

Epoch 04372: saving model to weights/music_vae-4372-0.04-17.77.h5


In [0]:
prediction_output = model.autoencoder.predict(songs_one_hot[np.newaxis, 0])

In [52]:
prediction_indices = np.argmax(prediction_output, axis=2)
code2note = dict([[code, note] for note, code in note2code.items()])

prediction_song = [code2note[index] for index in prediction_indices[0]]
print(prediction_song)

['5.11', '1.4', '2.5', '2.5', '4.7', '9.1', 'B-3', 'D5', 'E3', 'E5', 'F3', 'D5', 'C#5', 'G3', 'D5', 'E5', 'A3', 'F5', 'E5', 'F5', 'E5', 'F5', 'A2', 'E5', 'D5', 'D3', 'D5', 'C5', 'B-4', 'A4', '10.2', '0.4', '5.9', '7.10', '9', '7.10', '5.7', 'C4', 'E4', 'B-3', '0.2', 'C3', 'C5', 'B-4', 'F3', 'A4', 'G4', '3.9', '11.2', 'E-3', 'C5', 'B4', 'F3', 'C5', 'G3', 'E-3', 'F3', '3.9', '11.2', '0', 'B2', 'D5', 'C5', 'G#3', 'D5', 'G3', 'F3', 'E-3', '2.7', '9.0', '11.2', '0.3', '2.5', '3.7', '8.0', 'B2', 'D5', 'A2', '11.0', 'G2', 'B4', 'A4', 'G4', 'F5', 'G2', 'E-5', 'D5', '8.0', '7.10', '5.8', '7', 'G#2', 'F4', 'E-5', 'A2', 'D5', 'C5', '7.10', '5.9', '3.7', '7.9', 'B-2', 'F5', 'E-5', 'B2', 'D5', 'F5', '0.3', '2.7', '9.0', '11.2', '0.3', '2.5', '3.7', '8.0', 'D3', 'B4', 'A4', 'B4', 'C3', 'B2', 'D5', 'A2', 'G2', 'F3', 'E-3', 'F4', 'D3', 'C3', 'E-4', 'B-2', 'G#2', 'C5', 'G2', 'F2', 'D4', 'E-2', 'F2', 'C5', 'B4', 'G2', 'A4', 'B4', '0', '2.7', '0.3', '2.5', 'G3', 'E-5', 'D5', 'G#3', 'E-5', 'B-3', 'G3', 'G

In [0]:
from music21 import instrument, note, stream, chord

def create_midi(prediction_output, file_path):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp=file_path)

In [0]:
create_midi(prediction_song, 'test_vae_onehot_out4.midi')

In [0]:
from google.colab import files

files.download('test_vae_onehot_out4.midi')

In [0]:
decoder_input = np.array([[-0.0001, 0.0001]])
decoder_predicted = model.decoder.predict(decoder_input)

In [78]:
prediction_indices = np.argmax(decoder_predicted, axis=2)
code2note = dict([[code, note] for note, code in note2code.items()])

prediction_song = [code2note[index] for index in prediction_indices[0]]
print(prediction_song)

['0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6', '0.6'

In [0]:
files.download('weights/music_vae-4128-0.90-1.45.h5')